In [1]:
#################################
# filenames
#################################

number = 23

probs_file = 'newprobs'+str(number)+'.json'
filename3s = 'newprobs'+str(number)+'genre.json'
filename3se = 'newprobs'+str(number)+'e.txt'
histo_file = 'histo'+str(number)+'.txt'
url_dict_file = 'url.json'


##############################
# setup
##############################

import requests
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import random

tstart = time.time()
print(tstart)

api_url = 'https://api.hooktheory.com/v1/'
login = {"username": "jl8429",
         "password": ""}

r = requests.post(api_url+'users/auth', data=login)
#print('r', r)
#print(r.json())
'''
{'id': 136206, 'username': 'myUsername', 'activkey': 'myKey'}
'''

activkey = '0e11e1d65a021d0a883f309f813d74ad'
header = {"Authorization": "Bearer "+activkey}


#################################
# RELOAD url_dict
#################################

with open(url_dict_file, 'r') as f:
    url_dict = json.load(f)

big_dict = {}


#################################
# continue
#################################

#with open('chordProbs3new.json', "r") as f:
with open(probs_file, "r") as f:
    data3 = json.load(f)
f2 = open(filename3se, 'w')

j = 1
totals = str(len(data3))
print(totals)

histo = [0]*5000

driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")

for d in data3:
    chord1, chord2, chord3, chord4 = d['child_path'].split(",")
    mykey = str(chord1)+','+str(chord2)+','+str(chord3)+','+str(chord4)
    print(str(j)+'/'+totals, mykey)
    song_list = []
    
    # don't overwrite what we've done already
    if not big_dict.get(mykey, False):
        try:
            myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
            
            # slow down...
            time.sleep(1.1)
            if str(myData)=="<Response [429]>":
                #print("too many requests")
                time.sleep(30)
                myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
                if str(myData)=="<Response [429]>":
                    time.sleep(30)
                    print("too many requests")
            
            page = 1
            keepGoing = True
            #while myData.content and myData.content != b'No songs match this chord progressionnull':
            while keepGoing:
                # can't parse if it's this byte string
                if myData.content != b'No songs match this chord progressionnull':
                    #print('page', page)
                    # parse each song
                    try:
                        my_json = myData.json()
                        songs = len(my_json)
                        #print('songs len', songs)
                        if songs:
                            # for each song, pull the genre, add it to the url_dict and big_dict, then write it to the file
                            for i in range(songs):
                                url = my_json[i]['url'].split("#")[0]
                                #print(url)
                                # check if already in url_dict to save us from having to load the page slowly and parse
                                if url in url_dict:
                                    #print(url, url_dict[url])
                                    my_json[i]['genre'] = url_dict[url]
                                # if not in dict, we need to parse page and add to dict
                                else:
                                    #time.sleep(1.1)
                                    driver.get(url)
                                    html = driver.page_source
                                    soup = BeautifulSoup(html, 'html.parser')
                                    try:
                                        #genre = soup.find_all(['multiselect'])[0].find('a').contents[0]
                                        # some songs don't have a genre and some have multiple genres...
                                        genres = soup.find_all(['multiselect'])[0].find_all('a')
                                        gk = 0
                                        glist = []
                                        for g in genres:
                                            # .find_all() will return a lot of garbage, but the genres will have a length of 1
                                            if len(g) > 1:
                                                # if even the first is garbage, it has no genre
                                                if gk == 0:
                                                    my_json[i]['genre'] = ["Unknown"]
                                                    url_dict[url] = ["Unknown"]
                                                break
                                            else:
                                                glist.append(g.contents[0])
                                            gk += 1
                                        my_json[i]['genre'] = glist
                                        url_dict[url] = glist
                                    except:
                                        print('3. Unable to extract genre for', url)
                                        my_json[i]['genre'] = ["Unknown"]
                                        url_dict[url] = ["Unknown"]

                                    # this is not where this block belongs
                                    # nesting it under "else" above made failures generate "[]" and 
                                    #    only appended new songs to song_list, ignoring those already in url_dict
#                                     #driver.close()
#                                     # clean up and append song to list
#                                     my_json[i].pop('section', None)
#                                     my_json[i]['url'] = url
#                                     #f.write('\n'+str(my_json[i])+',')
#                                     song_list.append(my_json[i])

                                # this is where the above block really belongs 
                                #     so song_list gets every song, even if the url is in url_dict
                                #driver.close()
                                # clean up and append song to list
                                my_json[i].pop('section', None)
                                my_json[i]['url'] = url
                                #f.write('\n'+str(my_json[i])+',')
                                #print(my_json[i])
                                song_list.append(my_json[i])

                            if songs == 20:
                                #keepGoing = True
                                page += 1
                                myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                                # slow down...
                                time.sleep(1.1)
                                if str(myData)=="<Response [429]>":
                                    #print("too many requests")
                                    time.sleep(30)
                                    myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                                    if str(myData)=="<Response [429]>":
                                        time.sleep(30)
                                        print("too many requests")
                            else:
                                # update histogram of how many songs per chord progression
                                index = page*20 - 20 + songs
                                print('histo1', index)
                                histo[index] += 1
                                keepGoing = False
                        else:
                            # update histogram of how many songs per chord progression
                            index = page*20 - 20
                            print("histo2", index)
                            histo[index] += 1
                            keepGoing = False

                    except:
                        # for some reason, page=20 doesn't work on 1-5-6-4... there are other data corruptions too
                        print('2. Unable to extract for', chord1, chord2, chord3, chord4, "page", page)
                        f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
                        f2.close()
                        f2 = open(filename3se, 'a')

                        page += 1
                        myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                        # slow down...
                        time.sleep(1.1)
                        if str(myData)=="<Response [429]>":
                            #print("too many requests")
                            time.sleep(30)
                            myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                            if str(myData)=="<Response [429]>":
                                time.sleep(30)
                                print("too many requests")
                else:
                    print("No songs match this chord progression", mykey)
                    keepGoing = False

        except:
            print('1. Unable to extract for', chord1, chord2, chord3, chord4)
            f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
            f2.close()
            f2 = open(filename3se, 'a')

            # slow down
            time.sleep(1.1)
        
        big_dict[mykey] = song_list
    #f.close()
    #f = open(filename3s, 'a')
    #time.sleep(1.1)
    j += 1
    # restart the driver every 100 chord progressions
    if j%100==0:
        driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")
    
f2.close()


with open(filename3s, "w") as f:
    json.dump(big_dict, f)

tend = time.time()
print(tend)
#f.close()
print(tend-tstart)

print(histo[:100])
with open(histo_file, "w") as f:  
    f.write(str(histo))

with open(url_dict_file, "w") as f:
    json.dump(url_dict, f)

1615161273.9726295
500
1/500 564,5,564,5/2
histo1 1
2/500 3,2,565/6,4
histo1 1
3/500 3,4,1,57/5
histo1 1
4/500 5,6,3,66
histo1 1
5/500 2,b4,5,1
histo1 2
6/500 5/2,1,5/2,57/2
histo1 1
7/500 6,4,16,5
histo1 6
8/500 57/6,67,5,47
histo1 1
9/500 2,4,542,6
histo1 1
10/500 6,5,1,5/6
histo1 6
11/500 67,46,L264,67
histo1 1
12/500 2,1,L7,6
histo1 1
13/500 1,4,b4,3
histo1 1
14/500 265,165,47,265
histo1 1
15/500 1,4,365,26
histo1 1
16/500 4,b3,2,Y2
histo1 1
17/500 5,542,16,4
histo1 4
18/500 4,5,564,27
histo1 1
19/500 6,4,16,4
histo1 3
20/500 27,664,47,5
histo1 1
21/500 564,66,56/5,5/5
histo1 1
22/500 5/5,5,1,6
histo1 2
23/500 5,67,b565,56
histo1 1
24/500 5,47,36,6
histo1 1
25/500 b6,2,5,1
histo1 1
26/500 5/5,6,4,1
histo1 5
27/500 1,164,4,5/5
histo1 1
28/500 2,7/6,2,7/6
histo1 1
29/500 1,142,6,5/5
histo1 1
30/500 1,56,3,5
histo1 1
31/500 27,243,37,343
histo1 1
32/500 46,164,46,56
histo1 1
33/500 47,165,77/4,7/4
histo1 1
34/500 5,67,47,5
histo1 4
35/500 17,27,37,164
histo1 1
36/500 56/6,6,17,1
histo

histo1 1
288/500 46,b46,1,56
histo1 1
289/500 27,77,47/4,57
histo1 1
290/500 3,67,4,1
histo1 3
291/500 2,4,2,3
histo1 6
292/500 b6,b4,5,1
histo1 1
293/500 5,6,66,3
histo1 1
294/500 164,4,47,16
histo1 1
295/500 4,6,5,56
histo1 3
296/500 b7,D5,4,D5
histo1 1
297/500 17,27,5/6,67
histo1 1
298/500 57/5,b4,164,143
histo1 1
299/500 47,2,5,1
histo1 1
300/500 5/6,57/6,1,5
histo1 1
301/500 37,67,56,56/2
histo1 1
302/500 4,57,6,56/5
histo1 1
303/500 47,443,b47,b77
histo1 1
304/500 56,46,4,D3
histo1 1
305/500 4,16,1,16
histo1 5
306/500 46,1,2,3
histo1 1
307/500 1,2,77,4
histo1 3
308/500 6,7,5,47
histo1 1
309/500 3,M5,2,6
histo1 1
310/500 1,364,6,164
histo1 7
311/500 67,27,5,7/6
histo1 1
312/500 464,77,47/4,46
histo1 1
313/500 b4,2,b4,2
histo1 2
314/500 1,47,5,4
histo1 2
315/500 1,5/3,36,564/3
histo1 1
316/500 1,164,5/2,564/2
histo1 1
317/500 56,b7,6,1
histo1 1
318/500 164,542,16,564
histo1 1
319/500 664,5,b16,16
histo1 1
320/500 4,6,5,5/5
histo1 9
321/500 5,27,365,4
histo1 1
322/500 5,5/5,27,6
his

In [ ]:
##########################
# list errors here
##########################
100/500 164,4,16,5/5
2. Unable to extract for 164 4 16 5/5 page 1
histo2 20
b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'

106/500 27,1,5,6
2. Unable to extract for 27 1 5 6 page 1
histo2 20
b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'

273/500 2,6,36,6
2. Unable to extract for 2 6 36 6 page 1
histo2 20
b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'

329/500 542,16,564,1
2. Unable to extract for 542 16 564 1 page 1
histo2 20
b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'

In [5]:
##########################
# test errors here
##########################
mykey = '542,16,564,1'
page = 1
if page == 1:
    myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
else:
    myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
print(myData.content)
#print(len(myData.json()))
#print(myData.json())

b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'


In [6]:
# ##########################
# # fix histo here
# ##########################
# histo[20] -= 4
# print(histo[:100])
# with open(histo_file, "w") as f:  
#     f.write(str(histo))

[0, 360, 65, 24, 9, 5, 6, 5, 6, 6, 1, 1, 1, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
